In [6]:
import os
import psycopg2
from dotenv import load_dotenv
from sshtunnel import SSHTunnelForwarder
import time
import pandas as pd

In [2]:
# Load environment variables from .env file
load_dotenv("environment_variables.env")
psql_pw = os.getenv("PSQL_PW")

# Get data
## DB Connection

In [3]:
# Connect to AWS RDS instance through AWS EC2 instance

# SSH parameters
bastion_host = 'ec2-34-224-93-62.compute-1.amazonaws.com'
bastion_user = 'ec2-user'
bastion_key = 'C:\\Users\\detto\\Documents\\ec2-key-pair.pem'

# RDS parameters
rds_host = 'youtubeviewprediction.cd0c8oow2pnr.us-east-1.rds.amazonaws.com'
rds_user = 'postgres'
rds_password = psql_pw
rds_database = 'YouTubeViewPrediction'
rds_port = 5432

try:
    # Create an SSH tunnel
    tunnel = SSHTunnelForwarder(
        (bastion_host, 22),
        ssh_username=bastion_user,
        ssh_pkey=bastion_key,
        remote_bind_address=(rds_host, rds_port),
        local_bind_address=('localhost', 6543)  # Choose a local port for the tunnel
    )

    # Start the tunnel
    tunnel.start()

    # Connect to PostgreSQL through the tunnel
    conn = psycopg2.connect(
        database=rds_database,
        user=rds_user,
        password=rds_password,
        host=tunnel.local_bind_host,
        port=tunnel.local_bind_port
    )

    # Create a cursor object
    cursor = conn.cursor()

    # Execute a SQL query
    cursor.execute("SELECT version();")

    # Fetch result
    record = cursor.fetchone()
    print("You are connected to - ", record)

except (Exception, psycopg2.Error) as error:
    print("Error while connecting to PostgreSQL", error)
    tunnel.stop()

You are connected to -  ('PostgreSQL 16.1 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 7.3.1 20180712 (Red Hat 7.3.1-12), 64-bit',)


## Turn SQL tables in to Pandas dataframes

In [8]:
video_fact = pd.read_sql_query("SELECT * FROM video_fact", conn)
video_dim = pd.read_sql_query("SELECT * FROM video_dim", conn)
channel_fact = pd.read_sql_query("SELECT * FROM channel_fact", conn)
channel_dim = pd.read_sql_query("SELECT * FROM channel_dim", conn)
categories_dim = pd.read_sql_query("SELECT * FROM categories_dim", conn)

In [9]:
conn.close()
cursor.close()
tunnel.stop()

# EDA
## video_fact

In [14]:
video_fact.shape

(59219, 5)

In [11]:
video_fact.dtypes

collected_at    datetime64[ns]
video_id                object
num_views                int64
num_likes                int64
num_comments             int64
dtype: object

In [12]:
video_fact.describe()

,num_views,num_likes,num_comments
count,5.921900e+04,5.921900e+04,59219.000000
mean,2.979527e+06,1.176571e+05,7854.126649
std,1.162246e+07,5.399850e+05,30267.041495
min,3.034000e+04,0.000000e+00,0.000000
25%,4.690950e+05,1.299650e+04,979.000000
50%,9.611560e+05,3.158900e+04,2447.000000
75%,2.063594e+06,7.691300e+04,4984.000000
max,2.156450e+08,8.272778e+06,385935.000000


In [17]:
video_fact[["collected_at", "video_id"]].nunique()

collected_at    129
video_id        720
dtype: int64

In [2]:
video_fact.isna().sum()

NameError: name 'video_fact' is not defined

In [1]:
video_fact["video_id"].value_counts()

NameError: name 'video_fact' is not defined